In [1]:
%pip install pymilvus openai transformers torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
from pymilvus import connections, Collection, utility, FieldSchema, CollectionSchema, DataType
from openai import OpenAI
from dotenv import load_dotenv

import time

# --- 1. 설정값 ---

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("오류: OPENAI_API_KEY 환경 변수를 설정해주세요.")
    exit()
LLM_MODEL = os.getenv("LLM_MODEL", "gpt-4o-mini") # 사용할 OpenAI 모델, 기본값 설정

# 임베딩 모델 설정 (Milvus 저장 시 사용했던 모델과 동일해야 함)
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME", "klue/bert-base")
VECTOR_DIM = int(os.getenv("VECTOR_DIM", "768"))

# Milvus 설정
MILVUS_HOST = os.getenv("MILVUS_HOST", "localhost")
MILVUS_PORT = os.getenv("MILVUS_PORT", "19530")
# COLLECTION_NAME = "celltrion_embedding" # <<<< 검색할 컬렉션 이름
# 만약 여러 컬렉션을 검색해야 한다면 리스트로 관리
COLLECTION_NAMES = ["celltrion_embeddings", "news_embeddings"]

# 검색 설정
SEARCH_TOP_K = 20 # Milvus에서 검색할 상위 K개 결과
SEARCH_PARAMS = {
    "metric_type": "L2",
    "params": {"nprobe": 10} # IVF_FLAT 인덱스 사용 시 nprobe 값 조절
    # "params": {"ef": 64} # HNSW 인덱스 사용 시 ef 값 조절
}

# --- 2. 모델 및 토크나이저 로드 ---
print("Loading embedding model and tokenizer...")
try:
    tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
    embedding_model = AutoModel.from_pretrained(EMBEDDING_MODEL_NAME)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    embedding_model.to(device)
    embedding_model.eval() # 평가 모드
    print(f"Using device for embedding: {device}")
except Exception as e:
    print(f"Error loading embedding model: {e}")
    exit()

# --- 3. 유틸리티 함수 ---

def mean_pooling(model_output, attention_mask):
    """Mean Pooling 계산 함수"""
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_embedding(text):
    """주어진 텍스트의 임베딩 벡터를 반환 (NumPy Array)"""
    try:
        encoded_input = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt').to(device)
        with torch.no_grad():
            model_output = embedding_model(**encoded_input)
        embedding = mean_pooling(model_output, encoded_input['attention_mask'])
        return embedding.cpu().numpy().flatten() # NumPy 배열로 변환 후 flatten
    except Exception as e:
        print(f"Error getting embedding for text '{text[:50]}...': {e}")
        return None

# search_milvus 함수 정의
def search_milvus(query_vector, collection_names_list, top_k_total):
    """
    Milvus의 여러 컬렉션에서 관련성 높은 청크를 검색하고 결과를 병합합니다.
    (결과 처리 시 컬렉션별 필드 접근 분기 적용)
    """
    all_retrieved_chunks = []
    try:
        if not connections.has_connection("default"):
             connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)

        for c_name in collection_names_list:
            # --- 컬렉션 확인, 로드, Output Fields 정의 (이전과 동일) ---
            print(f"\nSearching in collection: '{c_name}'...")
            # ... (collection 존재 확인, load 로직 동일) ...
            collection = Collection(c_name)
            try:
                print(f"  Ensuring collection '{c_name}' is loaded...")
                collection.load()
                utility.wait_for_loading_complete(c_name)
                print(f"  Collection '{c_name}' is ready for search.")
            except Exception as load_err:
                print(f"  Error loading collection '{c_name}': {load_err}")
                continue

            current_output_fields = []
            text_field_for_this_collection = ""
            if c_name == "celltrion_embeddings":
                 text_field_for_this_collection = "text"
                 current_output_fields = [text_field_for_this_collection]
            elif c_name == "news_embeddings":
                 text_field_for_this_collection = "chunk_text"
                 current_output_fields = [
                     text_field_for_this_collection, "original_article_id",
                     "chunk_seq_id", "title", "datetime", "summary", "url"
                 ]
            else:
                 print(f"  Warning: Unknown collection '{c_name}'. Skipping search.")
                 continue

            # --- 검색 실행 (이전과 동일) ---
            try:
                search_results = collection.search(
                    data=[query_vector.tolist()],
                    anns_field="embedding",
                    param=SEARCH_PARAMS,
                    limit=top_k_total,
                    output_fields=current_output_fields
                )

                # --- 결과 처리 (★★ 컬렉션별 필드 접근 분기 적용 ★★) ---
                if search_results and search_results[0]:
                    for hit in search_results[0]:
                        try:
                            hit_id = hit.id
                            hit_score = hit.distance
                            entity = hit.entity # entity 객체 가져오기

                            # 공통 필드 및 기본값으로 chunk_data 초기화
                            chunk_data = {
                                "collection": c_name,
                                "id": hit_id,
                                "score": hit_score,
                                # 텍스트 필드는 항상 요청되므로 getattr 사용 가능
                                "text": getattr(entity, text_field_for_this_collection, ""),
                                # source_type 기본값 설정 (스키마에 없으므로)
                                "source_type": c_name
                            }

                            # ★★★ 컬렉션에 따라 추가 필드 접근 ★★★
                            if c_name == "news_embeddings":
                                # news_embeddings 스키마에 있고 output_fields로 요청한 필드만 접근
                                chunk_data["title"] = getattr(entity, "title", "")
                                chunk_data["url"] = getattr(entity, "url", "")
                                chunk_data["original_article_id"] = getattr(entity, "original_article_id", None)
                                chunk_data["chunk_seq_id"] = getattr(entity, "chunk_seq_id", None)
                                chunk_data["datetime"] = getattr(entity, "datetime", "")
                                chunk_data["summary"] = getattr(entity, "summary", "")
                                # 만약 news 스키마에 source_type이 있다면 여기서 덮어쓰기
                                # chunk_data["source_type"] = getattr(entity, "source_type", c_name)

                            elif c_name == "celltrion_embeddings":
                                # celltrion_embeddings 스키마에는 추가 메타데이터 필드가 없음
                                # 따라서 여기서 접근할 필드 없음
                                pass
                                # 만약 celltrion 스키마에 source_type이 있다면 여기서 덮어쓰기
                                # chunk_data["source_type"] = getattr(entity, "source_type", c_name)

                            all_retrieved_chunks.append(chunk_data)

                        except Exception as process_err:
                            # 개별 hit 처리 중 예외 발생 시 로그 남기고 계속
                            print(f"  Warning: Error processing hit {getattr(hit, 'id', 'N/A')} in {c_name}: {process_err}")
                            continue # 다음 hit으로 넘어감

                    print(f"  Found {len(search_results[0])} results in '{c_name}'.")

            except Exception as search_err:
                print(f"  Error searching collection '{c_name}': {search_err}")

        # --- 결과 취합 후 정렬 및 제한 (이하 동일) ---
        if all_retrieved_chunks:
            all_retrieved_chunks.sort(key=lambda x: x['score'])
            print(f"\nTotal results from all collections: {len(all_retrieved_chunks)}")
            final_chunks = all_retrieved_chunks[:top_k_total]
            print(f"Returning top {len(final_chunks)} overall results.")
            return final_chunks
        else:
            return []

    except Exception as e:
        print(f"Error during Milvus search process: {e}")
        return []

def format_context(retrieved_chunks):
    """검색된 청크들을 LLM 프롬프트에 넣기 좋은 형태의 문자열로 변환"""
    context_str = ""
    for i, chunk in enumerate(retrieved_chunks):
        context_str += f"--- 문서 {i+1} (ID: {chunk['id']}, 출처: {chunk.get('title', 'N/A')}) ---\n"
        context_str += chunk.get('text', '') + "\n\n"
    return context_str.strip()

def ask_llm(query, context):
    """LLM에게 질문과 컨텍스트를 전달하고 답변을 받음"""
    prompt = f"""
[배경 정보]

- 분석 대상 기업: 셀트리온
- 현재 시점: 2025년 1월 1일

[목표]

셀트리온의 24년 4분기 실적 분석 및 향후 전망에 대한 종합적인 기업 분석 보고서를 생성하는 것입니다.

[보고서 작성 가이드라인]

- 정보 출처 명확화: 보고서의 모든 내용은 오직 RAG를 통해 제공된 컨텍스트에만 근거해야 합니다. 컨텍스트에 명시적으로 언급되지 않은 외부 정보, 추측, 또는 개인적인 의견을 포함하지 마십시오.
- 객관성 유지: 사실에 기반하여 객관적이고 중립적인 톤으로 서술하십시오.
- 구조화: 아래 제시된 구조에 따라 정보를 논리적으로 구성하여 보고서를 작성하십시오.

[생성할 보고서의 구조 및 포함 내용 지침]

다음 구조를 따라 보고서를 작성하되, 각 섹션에 해당하는 내용을 제공된 컨텍스트(공시, 뉴스 기사)에서 찾아서 요약하고 기술하십시오.

1. 보고서 요약 (Executive Summary):
    - 컨텍스트에 기반한 셀트리온 4Q24 실적의 주요 특징 요약.
    - 컨텍스트에서 파악된 향후 사업 방향 또는 전망에 대한 핵심 내용 요약.
2. 2024년 4분기 실적 분석:
    - 실적 요인 분석: 공시 내용이나 뉴스 기사에서 언급된 4Q24 실적의 주요 변동 요인(긍정적/부정적 요인 모두)을 설명. (예: 특정 제품의 판매 호조/부진, 비용 증가/감소 요인, 일회성 손익 발생 등 공식적으로 언급된 내용)
3. 주요 사업 및 제품 동향:
    - 제품 관련 소식: 컨텍스트에서 언급된 주요 제품(예: 램시마SC, 유플라이마, 베그젤마, 짐펜트라, 스테키마 등) 관련 최신 동향(예: 주요 시장 출시/허가 현황, 판매 관련 언급, 생산 관련 소식 등)을 요약.
    - R&D 및 파이프라인: 컨텍스트에서 찾을 수 있는 신약 개발 진행 상황, 임상 결과 발표, 기술 도입 등 R&D 관련 중요 업데이트 사항을 기술.
    - CMO 사업: 위탁생산(CMO) 관련 계약, 생산 등 컨텍스트 내 관련 정보를 요약.
    - 기타 사업: 합병 관련 진행 상황 및 시너지 창출 노력 등 컨텍스트 내 기타 중요 사업 내용을 포함.
4. 시장 환경 및 전략 방향:
    - 주요 시장 활동: 컨텍스트에 나타난 주요 시장(예: 미국, 유럽)에서의 활동 내용(예: 신제품 출시, 허가 신청, 마케팅 활동, 시장 경쟁 관련 언급 등)을 요약.
    - 회사의 공식 전략: 컨텍스트의 공시나 보도자료 등에서 발표된 회사의 주요 경영 전략, 투자 계획, 파트너십 체결 등 내용을 정리.
5. 향후 전망 (공식 발표 기반):
    - 회사의 공식 입장/계획: 컨텍스트에서 확인되는 2025년 사업 계획, 목표, 신제품 출시 예정, 성장 전략 등 회사에서 공식적으로 발표한 향후 전망 관련 내용을 요약. (애널리스트의 예측이 아닌, 회사의 발표 내용 중심)
    - 미래 성과 영향 요인: 컨텍스트 정보를 바탕으로 향후 실적에 영향을 미칠 수 있는 주요 요인(예: 신제품 성과 기대, 진행 중인 R&D 중요성, 시장 환경 변화 등 공식 발표나 뉴스에서 강조된 내용)을 정리.
6. 기타 참고사항:
    - 컨텍스트(공시, 뉴스 기사)에서 언급된 기타 중요 정보, 잠재적 위험 요인 또는 기회 요인 등을 객관적으로 요약. (투자 추천이나 가치 판단은 제외)

[보고서 평가 기준]

평가 목표: 생성된 보고서가 주어진 컨텍스트(셀트리온 4Q24 공시 자료 및 관련 뉴스 기사)의 정보를 얼마나 정확하고 충실하게, 그리고 구조적으로 잘 요약했는지 평가합니다.

종합 평가 기준 (모든 목차 공통 적용):

1. 컨텍스트 충실성 (Context Fidelity): 보고서의 모든 내용이 오직 제공된 컨텍스트 정보에만 기반하는가? 외부 정보나 환각(Hallucination)은 없는가? (가장 중요)
2. 구조 준수성 (Structural Adherence): 제시된 6가지 목차 구조를 정확히 따르고 있는가?
3. 객관성 및 톤 (Objectivity & Tone): 보고서 전체적으로 객관적이고 사실 기반의 중립적인 톤을 유지하는가? 추측이나 주관적 평가는 배제되었는가?
4. 명확성 및 가독성 (Clarity & Readability): 사용된 언어가 명확하고 이해하기 쉬운가? 정보가 각 목차 내에서 논리적으로 구성되어 있는가?

목차별 세부 평가 기준:

1. 보고서 요약 (Executive Summary)

- 핵심 내용 반영도: 보고서 본문(2~6번 목차)의 핵심 내용(4Q24 실적 주요 특징, 향후 전망 핵심)을 정확하게 요약하고 있는가?
- 정확성 및 일관성: 요약된 내용이 컨텍스트 및 보고서 본문의 내용과 일치하며 왜곡이 없는가?
- 간결성: 핵심 내용을 간결하게 전달하는가? 불필요하게 상세하지 않은가?
- 포괄성: 실적 측면과 전망 측면을 균형 있게 포함하는가?

2. 2024년 4분기 실적 분석 (4Q24 Performance Review)

- 실적 요인 분석 정확성: 실적 변동 요인(매출 동인, 비용 요인 등) 설명이 컨텍스트(공시, 뉴스) 내용과 정확히 일치하는가?
- 실적 요인 분석 완전성: 컨텍스트에서 언급된 *중요한* 실적 변동 요인을 충분히 다루고 있는가?
- 컨텍스트 기반: 분석 내용이 컨텍스트 정보에만 근거하며 외부 해석이 배제되었는가?

3. 주요 사업 및 제품 동향 (Business & Product Developments)

- 정보 정확성: 제품 동향, R&D 업데이트, CMO 관련 기술 내용이 컨텍스트 정보와 사실적으로 일치하는가?
- 정보 완전성: 컨텍스트에서 언급된 주요 제품, R&D, CMO 관련 *중요* 업데이트 사항을 누락 없이 포함했는가?
- 관련성: 보고서 목차의 주제(사업 및 제품 동향)와 관련된 내용을 충실히 담고 있는가?
- 컨텍스트 기반: 내용이 컨텍스트(공시, 뉴스)에서 직접 확인 가능한 정보인가?

4. 시장 환경 및 전략 방향 (Market Environment & Strategy)

- 정보 정확성: 주요 시장(미국, 유럽 등) 활동 및 회사 전략(합병 시너지, 파트너십 등)에 대한 설명이 컨텍스트 정보와 일치하는가?
- 정보 완전성: 컨텍스트에서 강조된 주요 시장 활동 및 전략 방향을 충분히 포함하고 있는가?
- 관련성: 내용이 시장 환경 및 회사의 전략적 움직임에 초점을 맞추고 있는가?
- 컨텍스트 기반: 설명이 컨텍스트에서 제공된 정보의 범위를 벗어나지 않는가?

5. 향후 전망 (공식 발표 기반) (Future Outlook - Based on Official Statements)

- 공식 입장 정확성: 회사의 공식적인 향후 계획, 목표, 신제품 출시 일정 등을 컨텍스트 내용 그대로 정확하게 전달하는가?
- 공식 입장 완전성: 컨텍스트에서 언급된 회사의 *주요* 공식 전망 내용을 포함하고 있는가?
- 출처 명확성: 해당 내용이 회사의 '공식 발표'에 기반한 것임을 명확히 인지할 수 있게 서술되었는가? (추측성 서술과 구분되는가?)
- 컨텍스트 기반: 회사의 공식 발표 내용을 왜곡하거나 과장하지 않았는가?

6. 기타 참고사항 (Key Considerations - Based on Public Info)

- 정보 정확성: 언급된 참고사항(리스크, 기회 등)이 컨텍스트 정보에 사실적으로 기반하는가?
- 정보 완전성: 컨텍스트 내 다른 목차에 포함되지 않은 *중요한* 기타 정보, 리스크, 기회 요인을 포함하는가?
- 객관성: 투자 추천이나 주관적 가치 판단 없이, 사실 정보를 객관적으로 전달하는가?
- 관련성: 내용이 '기타 참고사항'으로서의 관련성을 가지는가?
- 컨텍스트 기반: 내용이 컨텍스트에서 직접 확인 가능한 정보인가?


[컨텍스트 정보]
{context if context else "제공된 컨텍스트 정보가 없습니다."}

[사용자 질문]
{query}

[답변]
"""

    try:
        client = OpenAI(api_key=OPENAI_API_KEY)
        response = client.chat.completions.create(
            model=LLM_MODEL,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context in Korean."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7, # 답변의 창의성 조절 (0에 가까울수록 결정적)
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error calling OpenAI API: {e}")
        return "OpenAI API 호출 중 오류가 발생했습니다."

# --- 4. RAG 파이프라인 실행 ---
if __name__ == "__main__":
    while True:
        user_query = input("\n질문을 입력하세요 (종료하려면 'exit' 입력): ")
        if user_query.lower() == 'exit':
            break
        if not user_query:
            continue

        # 1. 쿼리 임베딩
        print("Embedding your query...")
        start_embed_time = time.time()
        query_embedding = get_embedding(user_query)
        embed_time = time.time() - start_embed_time
        if query_embedding is None:
            print("쿼리 임베딩 중 오류가 발생했습니다.")
            continue
        print(f"Query embedding done ({embed_time:.2f}s)")

        # 2. Milvus 검색

        # 2. Milvus 검색 (수정됨: 함수 호출 방식 변경)
        print(f"Searching Milvus collections {COLLECTION_NAMES} for top {SEARCH_TOP_K} relevant chunks overall...")
        start_search_time = time.time()
        # 수정된 함수 호출: 컬렉션 이름 리스트와 최종 원하는 결과 개수 전달
        retrieved_data = search_milvus(query_embedding, COLLECTION_NAMES, SEARCH_TOP_K)
        search_time = time.time() - start_search_time

        if not retrieved_data:
            print("Milvus에서 관련 정보를 찾지 못했습니다.")
            context_for_llm = ""
        else:
            # 이제 retrieved_data는 여러 컬렉션의 결과를 포함하고 score 기준으로 정렬됨
            print(f"Found {len(retrieved_data)} relevant chunks overall ({search_time:.2f}s).")
            # (선택적) 검색 결과 미리보기 (출처 컬렉션 포함)
            # for i, chunk in enumerate(retrieved_data):
            #     print(f"  Result {i+1}: Score={chunk['score']:.4f}, Collection='{chunk['collection']}', Text={chunk['text'][:80]}...")
            context_for_llm = format_context(retrieved_data) # format_context 함수는 그대로 사용 가능

        # 3. LLM에게 질문/답변 생성
        print("Asking LLM...")
        start_llm_time = time.time()
        answer = ask_llm(user_query, context_for_llm)
        llm_time = time.time() - start_llm_time
        print(f"LLM response received ({llm_time:.2f}s).")

        # 4. 답변 출력
        print("\n[답변]")
        print(answer)

    print("RAG 시스템을 종료합니다.")

Loading embedding model and tokenizer...
Using device for embedding: cpu
RAG 시스템을 종료합니다.


In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI API 키를 환경 변수에서 가져옴
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("오류: OPENAI_API_KEY 환경 변수를 설정해주세요.")
    exit()

client = OpenAI(
  api_key=OPENAI_API_KEY
)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  store=True,
  messages=[
    {"role": "user", "content": "셀트리온의 최근 1년간 재무 성과와 주요 바이오시밀러 매출 동향 보고서 작성해줘. 특히 컨텍스트 정보를 바탕으로 셀트리온의 신규 개발 현황 요약을 포함해줘. 그리고 셀트리온의 사업 전략과 향후 목표에 대한 설명도 포함해줘. "}
  ]
)

print(completion.choices[0].message)


ChatCompletionMessage(content='### 셀트리온 최근 1년간 재무 성과 및 주요 바이오시밀러 매출 동향 보고서\n\n#### 1. 재무 성과 개요\n\n셀트리온은 2022년과 2023년 동안 지속적인 성장을 보여주었습니다. 2023년 3분기 발표된 재무 보고서에 따르면, 전년 동기 대비 매출과 순이익이 증가하였습니다. 구체적인 수치는 아직 공개되지 않았지만, 일반적으로 바이오시밀러 시장의 성장과 함께 셀트리온의 매출이 호조를 보였습니다.\n\n- **매출 성장**: 셀트리온의 바이오시밀러 제품군에서 매출 증가가 두드러지며, 특히 유럽과 아시아 지역에서의 판매가 크게 증가했습니다.\n- **순이익**: 운영 효율성을 통해 경상이익이 증가하는 추세를 보였습니다. 연구 개발(R&D) 투자에도 불구하고 마진이 개선되었습니다.\n\n#### 2. 주요 바이오시밀러 매출 동향\n\n셀트리온의 주력 바이오시밀러 제품군에는 다음과 같은 약물이 포함됩니다:\n\n- **트룩시마(Truxima)**: 리툭시맙의 바이오시밀러로, hematologic malignancies 및 자가면역 질환 치료에 사용됩니다. 유럽, 대한민국에서의 시장 점유율 증가로 매출이 상승하였습니다.\n- **허쥬마(Herceptin)**: 트라스투주맙의 바이오시밀러로, 유방암 및 위암 치료에 사용됩니다. 글로벌 승인 확대와 판매 네트워크 강화로 매출이 증가했습니다.\n- **유플루자(Infliximab)**: 인플릭시맵의 바이오시밀러로, 자가면역 질환에 대한 수요 증가로 매출이 상승했습니다.\n\n이에 따라, 셀트리온은 바이오시밀러 시장에서의 경쟁력을 지속적으로 확보하고 있습니다.\n\n#### 3. 신규 개발 현황 요약\n\n셀트리온은 끊임없는 연구 개발을 통해 신규 바이오 의약품 및 치료제를 개발 중입니다. 최근 몇 가지 주요 개발 사항은 다음과 같습니다:\n\n- **면역항암제 개발**: 셀트리온은 면역항암제의 개발을 확대하고 있으며, 여러 임상시험 단계에 있는 후보물질이 있습